## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Balkhash,KZ,46.8440,74.9804,22.75,83,100,6.13,light snow
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,90,95,4.00,overcast clouds
2,2,Klaksvik,FO,62.2266,-6.5890,38.32,98,100,31.74,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.86,73,12,3.76,few clouds
4,4,Chuy,UY,-33.6971,-53.4616,78.44,52,17,15.26,few clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,90,95,4.00,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.86,73,12,3.76,few clouds
4,4,Chuy,UY,-33.6971,-53.4616,78.44,52,17,15.26,few clouds
9,9,Tabou,CI,4.4230,-7.3528,80.13,82,93,7.54,overcast clouds
12,12,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,89,40,5.75,heavy intensity rain
16,16,Bengkulu,ID,-3.8004,102.2655,75.31,90,93,3.74,overcast clouds
17,17,Buala,SB,-8.1450,159.5921,80.56,83,100,3.91,light rain
25,25,Hilo,US,19.7297,-155.0900,78.12,62,20,0.00,light rain
26,26,Salinopolis,BR,-0.6136,-47.3561,83.84,71,8,12.26,clear sky
31,31,Sao Filipe,CV,14.8961,-24.4956,76.64,68,0,6.58,clear sky


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                209
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df =  preferred_cities_df.dropna()
clean_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
3,Rikitea,PF,77.86,few clouds,-23.1203,-134.9692,
4,Chuy,UY,78.44,few clouds,-33.6971,-53.4616,
9,Tabou,CI,80.13,overcast clouds,4.4230,-7.3528,
12,Arraial Do Cabo,BR,78.76,heavy intensity rain,-22.9661,-42.0278,
16,Bengkulu,ID,75.31,overcast clouds,-3.8004,102.2655,
17,Buala,SB,80.56,light rain,-8.1450,159.5921,
25,Hilo,US,78.12,light rain,19.7297,-155.0900,
26,Salinopolis,BR,83.84,clear sky,-0.6136,-47.3561,
31,Sao Filipe,CV,76.64,clear sky,14.8961,-24.4956,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

no_hotels = []

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found near {city, lat, lng}... skipping.")
        no_hotels.append(city)
print(f"There were {len(no_hotels)} cities without hotels.\nThey included: {no_hotels}.")
        

Hotel not found near ('Morros', -9.4467, -46.3003)... skipping.
Hotel not found near ('Machinga', -15.1685, 35.3)... skipping.
Hotel not found near ('Dibombari', 4.1787, 9.6561)... skipping.
Hotel not found near ('Bowmore', 34.9421, -79.3031)... skipping.
Hotel not found near ('Rafai', 4.95, 23.9167)... skipping.
Hotel not found near ('Khudumelapye', -23.8833, 24.75)... skipping.
Hotel not found near ('Avera', 33.194, -82.5271)... skipping.
Hotel not found near ('Mitu', 1.1983, -70.1733)... skipping.
Hotel not found near ('Esperantinopolis', -4.8667, -44.7083)... skipping.
Hotel not found near ('Pangoa', -12.1167, -73.0)... skipping.
Hotel not found near ('Gogrial', 8.5332, 28.1004)... skipping.
There were 11 cities without hotels.
They included: ['Morros', 'Machinga', 'Dibombari', 'Bowmore', 'Rafai', 'Khudumelapye', 'Avera', 'Mitu', 'Esperantinopolis', 'Pangoa', 'Gogrial'].


In [76]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Rikitea,PF,77.86,few clouds,-23.1203,-134.9692,People ThankYou
4,Chuy,UY,78.44,few clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
9,Tabou,CI,80.13,overcast clouds,4.4230,-7.3528,hotêl doufoulougou
12,Arraial Do Cabo,BR,78.76,heavy intensity rain,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
664,Karonga,MW,80.19,scattered clouds,-9.9333,33.9333,Sumuka Inn
671,Belmonte,BR,81.10,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
672,Kota Kinabalu,MY,76.98,few clouds,5.9749,116.0724,Gaya Centre Hotel
690,Kudat,MY,77.68,overcast clouds,6.8837,116.8477,Dream Garden Hotel


In [72]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [78]:
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]

In [80]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [81]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [82]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))